__Step 1 :__ importer request, pandas, numpy et matplot

In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

__Step 2 :__ requête sur overpass avec output json, nodes dans area défini par un Wikidata ID == hydrater

In [29]:
wikidata = 'Q1664007'
r_url = '%5Bout%3Ajson%5D%3B%0D%0Aarea%5Bwikidata%3D'+wikidata+'%5D%3B%0D%0Anode%28area%29%5B%7E%27%5Ba-z0-9%5D%27%7E%27%5Ba-z0-9%5D%27%5D%3B%0D%0Aout%3B'
r = requests.get('https://www.overpass-api.de/api/interpreter?data='+r_url)
jsonobject = r.json()
elements = jsonobject['elements']

In [3]:
'''df = pd.DataFrame(relations['elements'])
df = df.drop(columns=['type','id'])
dict = df.to_dict()
dict = dict['tags']'''

"df = pd.DataFrame(relations['elements'])\ndf = df.drop(columns=['type','id'])\ndict = df.to_dict()\ndict = dict['tags']"

In [30]:
len(elements)

3386

In [31]:
table = list()
for element in elements: # elements isolé
    for tag, value in element['tags'].items():
        table.append([element['id'],element['lat'],element['lon'],element['type'],tag,value])

In [32]:
df = pd.DataFrame(table,columns=['id','lat','lon','type','tag','value'])

__Step 3 :__ statistique sur la fréquence des tags

In [38]:
print(len(df['tag'].value_counts()))
c = df['tag'].value_counts()[20:40]
c

365


ref                      196
addr:suburb              195
emergency                184
fire_hydrant:diameter    171
fire_hydrant:type        170
leaf_type                154
leaf_cycle               147
level                    143
highway                  124
man_made                 118
addr:floor               117
contact:website           97
surveillance:type         97
cuisine                   78
contact:facebook          77
barrier                   76
public_transport          67
note                      66
currency:CHF              66
currency:EUR              65
Name: tag, dtype: int64

__Step 4 :__ filtre spécifique, écriture de fonction
- filtre. entrée : colonne, valeur. Correspondance avec ID
- idée : fonction qui enlève éléments qui a un certain Tag

In [18]:
def selectClusters(df, column='tag', value='name'):
    idswithavalueincolumn = df[df[column]==value]['id']
    filtered = df[df['id'].isin(idswithavalueincolumn)]
    return filtered[filtered[column] != value]

__-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------__

In [49]:
df[df['tag']=='amenity']['value'].value_counts()

restaurant                120
bench                      80
telephone                  38
waste_basket               33
bar                        32
cafe                       30
fountain                   28
atm                        28
vending_machine            24
fast_food                  23
post_box                   21
bank                       19
bicycle_parking            16
pharmacy                   15
embassy                    11
nightclub                   9
toilets                     9
cinema                      9
bicycle_rental              9
theatre                     7
pub                         7
lounger                     6
place_of_worship            6
parking                     5
library                     5
childcare                   4
dentist                     4
post_office                 4
doctors                     4
social_facility             4
school                      4
car_sharing                 3
clock                       3
parking_en

In [50]:
selectClusters(df,'value','fountain')['tag'].value_counts()

wikimedia_commons          28
image                      28
source                     27
drinking_water             26
name                       21
operator                   21
wikidata                   19
location:description:de    19
brand                      17
description:de             11
alt_name                    8
note                        7
name:description:de         7
owner                       6
wikipedia                   6
name:de                     3
layer                       2
ref:kgs                     2
artist:wikidata             2
name:fr                     2
covered                     2
name:ru                     2
loc_name                    1
name:it                     1
old_name                    1
name:es                     1
tourism                     1
historic                    1
start_date                  1
name:en                     1
indoor                      1
Name: tag, dtype: int64

In [42]:
selectClusters(df,'tag','historic')[selectClusters(df,'tag','historic')['tag']=='name']

,id,lat,lon,type,tag,value
158,59632661,46.945854,7.440294,node,name,Weltpostdenkmal
1995,670501812,46.948733,7.457036,node,name,Zähringer-Denkmal
3988,1296812925,46.949209,7.448097,node,name,Erlach-Denkmal
9288,2493772791,46.945493,7.440594,node,name,Bider-Denkmal
22470,5463087118,46.944756,7.440502,node,name,Niggeler-Denkmal
23112,5543349328,46.948865,7.452926,node,name,Lenbrunnen
